In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import ast

from multiprocessing import Pool

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
secuencias_entrenamiento_df = pd.read_csv('../../Data/secuencias_entrenamiento.csv')
secuencias_entrenamiento_df

,session_id,item_id_salida,secuencia_items_ids,secuencia_fechas
0,3,15085,"[9655, 9655]","['2020-12-18 21:19:48.093', '2020-12-18 21:25:..."
1,13,18626,[15654],['2020-03-13 19:35:27.136']
2,18,24911,"[4026, 2507, 18316]","['2020-08-26 19:15:47.232', '2020-08-26 19:16:..."
3,19,12534,"[19896, 27937, 12804, 25772, 6341, 23687, 8316...","['2020-11-02 16:30:36.378', '2020-11-02 16:30:..."
4,24,13226,"[2927, 2927, 16064, 11662, 434, 18539, 10414, ...","['2020-02-26 17:22:48.903', '2020-02-26 17:23:..."
...,...,...,...,...
999995,4439986,2915,"[23502, 13914, 13403, 19310, 12373, 27733]","['2021-05-13 11:46:56.606', '2021-05-13 11:48:..."
999996,4439990,8786,"[22093, 17429, 1389, 1131, 4062, 26011, 4028, ...","['2020-08-22 11:38:48.785', '2020-08-22 11:39:..."
999997,4439994,21630,[25357],['2020-11-27 20:08:37.262']
999998,4439999,16962,"[6007, 8502, 13056, 2173, 8433, 1299, 15853]","['2020-11-27 10:52:12.577', '2020-11-27 10:53:..."


In [4]:
encoded_items_df = pd.read_csv('../../Data/items_item2vec_encoded.csv')
encoded_items_df

,item_id,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
0,2,0.000000e+00,0.000000e+00,8.259485e-38,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,...,1.147871e-04,1.196613e-25,0.000000,0.000000e+00,4.975298e-21,0.000000,1.000000e+00,1.283712e-37,0.000000e+00,5.708601e-11
1,3,7.226150e-16,2.417073e-01,2.325430e-14,3.434116e-10,9.999780e-01,2.084914e-19,2.359839e-04,7.360519e-18,3.367935e-12,...,6.631301e-28,1.442925e-15,1.000000,7.131817e-04,9.999999e-01,0.034609,8.714515e-01,5.728165e-07,1.033759e-04,9.978011e-01
2,4,7.434464e-30,7.750213e-18,0.000000e+00,1.445317e-29,1.000000e+00,0.000000e+00,0.000000e+00,3.267208e-25,9.452282e-33,...,0.000000e+00,5.438013e-28,1.000000,5.429324e-03,1.000000e+00,1.000000,9.924654e-01,6.889178e-05,9.211239e-23,2.146675e-14
3,7,5.490076e-37,0.000000e+00,1.191505e-21,9.322182e-12,4.260253e-13,5.699080e-01,1.000000e+00,4.086731e-30,5.050279e-12,...,1.000000e+00,0.000000e+00,0.000000,1.472102e-12,4.147344e-34,0.000000,0.000000e+00,1.565254e-36,1.000000e+00,1.000000e+00
4,8,2.423661e-25,2.795917e-11,6.869838e-35,1.957358e-24,1.000000e+00,1.214310e-37,0.000000e+00,2.739703e-17,1.734631e-19,...,0.000000e+00,1.350020e-19,1.000000,7.157927e-10,1.000000e+00,1.000000,2.604076e-02,1.155714e-02,3.667003e-09,1.811880e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23686,28139,9.726211e-01,2.284531e-11,2.776817e-11,5.597994e-02,1.026659e-02,2.461190e-02,1.000000e+00,3.444716e-18,1.398369e-05,...,4.171538e-03,1.981965e-04,0.961271,7.362516e-17,3.345782e-01,0.000326,3.236383e-25,6.360826e-16,9.999509e-01,9.999608e-01
23687,28140,9.997767e-01,2.538743e-08,7.991548e-11,2.896282e-21,3.695540e-04,9.091385e-18,2.061393e-09,1.558076e-14,5.036157e-05,...,3.120203e-29,1.000000e+00,1.000000,2.877177e-09,2.796884e-04,0.615862,9.811885e-01,1.042302e-14,2.447064e-09,9.952277e-01
23688,28141,2.941844e-15,7.376524e-11,3.965195e-17,8.104620e-05,9.995428e-01,5.364074e-11,4.411794e-07,4.666649e-08,1.239170e-11,...,2.487879e-37,3.125461e-19,1.000000,4.307819e-08,1.000000e+00,0.987323,9.997515e-01,3.231751e-02,1.217761e-02,8.776583e-06
23689,28142,0.000000e+00,1.112993e-36,1.000000e+00,1.000000e+00,3.748123e-22,1.000000e+00,1.000000e+00,1.000000e+00,1.470198e-26,...,1.000000e+00,0.000000e+00,0.000000,5.781284e-18,0.000000e+00,0.000000,6.874122e-17,0.000000e+00,1.000000e+00,5.961677e-13


## Item id objetivos de los datos de validacion, los ultimos 100000

In [5]:
target_sessions_valdata = list(secuencias_entrenamiento_df['session_id'])[-100000:]
print(len(target_sessions_valdata))
print(target_sessions_valdata[-5:])

100000
[4439986, 4439990, 4439994, 4439999, 4440001]


In [6]:
target_ids_valdata = list(secuencias_entrenamiento_df['item_id_salida'])[-100000:]
print(len(target_ids_valdata))
print(target_ids_valdata[-5:])

100000
[2915, 8786, 21630, 16962, 16631]


In [7]:
predicted_latent_items = np.load('../../Data/predicted_val_biLSTM_latent_items.npy')
print(predicted_latent_items.shape)
print(predicted_latent_items[0])

(100000, 64)
[0.03001219 0.02285046 0.11123247 0.3893673  0.885809   0.36846858
 0.43109182 0.7777968  0.06636418 0.8426795  0.9676862  0.6634793
 0.20819728 0.7969094  0.02610815 0.9668684  0.08637843 0.65192944
 0.3467252  0.12184291 0.91704476 0.03143597 0.01666406 0.81639236
 0.24290389 0.321087   0.8651691  0.8158176  0.96912706 0.01587688
 0.3704735  0.15070349 0.42914706 0.5896295  0.12340828 0.8538756
 0.7936835  0.5563243  0.5276125  0.9545258  0.54506606 0.17719251
 0.6487162  0.43461093 0.82679695 0.929219   0.5523493  0.03001323
 0.13274595 0.06090919 0.01068219 0.32276946 0.93914235 0.04661961
 0.27612847 0.44470787 0.91887474 0.1443953  0.962184   0.7319624
 0.9276975  0.2174379  0.13556367 0.53428376]


In [8]:
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
test_item = predicted_latent_items[0:1,:]
print(test_item.shape)

encoded_items_tensor = encoded_items_df.drop('item_id', axis=1).to_numpy()
print(encoded_items_tensor.shape)

test_result_cosine_similarity = cosine_similarity(test_item, encoded_items_tensor)
print(test_result_cosine_similarity.shape)

(1, 64)
(23691, 64)
(1, 23691)


In [10]:
test_result_cosine_similarity

array([[0.45152147, 0.78177445, 0.74269655, ..., 0.83281182, 0.2230114 ,
        0.38820933]])

In [41]:
print(predicted_latent_items.shape)
print(encoded_items_tensor.shape)

test_results_cosine_similarity = cosine_similarity(predicted_latent_items, encoded_items_tensor)
print(test_results_cosine_similarity.shape)

(100000, 64)
(23691, 64)
(100000, 23691)


In [44]:
test_results_cosine_similarity[0]

array([0.45152147, 0.78177445, 0.74269655, ..., 0.83281182, 0.2230114 ,
       0.38820933])

In [17]:
ids_items_encoded_list = list(encoded_items_df['item_id'])

In [18]:
restult_test_cosine_similarity = list(test_result_cosine_similarity[0])

In [19]:
restult_test_cosine_similarity_items_ids = pd.DataFrame(
    {'item_id': ids_items_encoded_list,
     'cosine_similarity': restult_test_cosine_similarity
    })

restult_test_cosine_similarity_items_ids

,item_id,cosine_similarity
0,2,0.451521
1,3,0.781774
2,4,0.742697
3,7,0.197067
4,8,0.721799
...,...,...
23686,28139,0.599884
23687,28140,0.715994
23688,28141,0.832812
23689,28142,0.223011


In [20]:
restult_test_cosine_similarity_items_ids.sort_values(by='cosine_similarity',ascending=False, inplace=True)
restult_test_cosine_similarity_items_ids

,item_id,cosine_similarity
5291,6292,0.952629
4250,5056,0.947155
21322,25273,0.947155
13017,15441,0.943601
18413,21809,0.942837
...,...,...
12538,14891,0.028497
11828,14039,0.027855
13932,16524,0.027741
12627,14995,0.026385


In [23]:
items_organizados = list(restult_test_cosine_similarity_items_ids['item_id'])

In [30]:
items_organizados.index(target_ids_valdata[0])

55

In [31]:
1/55

0.01818181818181818

In [70]:
def get_rank_of_prediction(target_item_id, cosine_similarity_list, items_ids_list):
    
    cosine_similarity_items_ids = pd.DataFrame(
    {
        'item_id': items_ids_list,
        'cosine_similarity': cosine_similarity_list
    })
    
    cosine_similarity_items_ids.sort_values(by='cosine_similarity',ascending=False, inplace=True)
    
    items_ids_top = list(cosine_similarity_items_ids['item_id'])
    
    rank = items_ids_top.index(target_item_id) + 1
    
    return rank

In [71]:
get_rank_of_prediction(target_ids_valdata[0], restult_test_cosine_similarity, ids_items_encoded_list)

56

In [48]:
encoded_items_tensor = encoded_items_df.drop('item_id', axis=1).to_numpy()
ids_items_encoded_list = list(encoded_items_df['item_id'])


ranks_predictions = []

for i in tqdm(range(predicted_latent_items.shape[0])):
    
    predicted_latent = np.expand_dims(predicted_latent_items[i], axis=0)
    
    cosine_similarity_result = cosine_similarity(predicted_latent, encoded_items_tensor)

    cosine_similarity_result_list = list(cosine_similarity_result[0])
    
    rank_prediction = get_rank_of_prediction(target_ids_valdata[i], cosine_similarity_result_list, ids_items_encoded_list)
    ranks_predictions.append(rank_prediction)

100%|██████████| 100000/100000 [35:41<00:00, 46.69it/s]


ranks_predictions = list(np.array(ranks_predictions) + 1)

In [66]:
val_predicitons_MMR = pd.DataFrame(
    {'target_sessions_valdata': target_sessions_valdata,
     'rank_prediction': ranks_predictions
    })

val_predicitons_MMR

,target_sessions_valdata,rank_prediction
0,3996722,56
1,3996725,5391
2,3996727,2972
3,3996729,27
4,3996731,1185
...,...,...
99995,4439986,84
99996,4439990,5566
99997,4439994,35
99998,4439999,1143


In [67]:
val_predicitons_MMR['reciprocal_rank_prediction'] = val_predicitons_MMR.apply(lambda row: 1 / row['rank_prediction'], axis=1)

In [68]:
val_predicitons_MMR

,target_sessions_valdata,rank_prediction,reciprocal_rank_prediction
0,3996722,56,0.017857
1,3996725,5391,0.000185
2,3996727,2972,0.000336
3,3996729,27,0.037037
4,3996731,1185,0.000844
...,...,...,...
99995,4439986,84,0.011905
99996,4439990,5566,0.000180
99997,4439994,35,0.028571
99998,4439999,1143,0.000875


In [69]:
val_predicitons_MMR.describe()

,target_sessions_valdata,rank_prediction,reciprocal_rank_prediction
count,1.000000e+05,100000.000000,100000.000000
mean,4.218551e+06,1398.559400,0.018487
std,1.280186e+05,2051.507685,0.080321
min,3.996722e+06,1.000000,0.000042
25%,4.107678e+06,132.000000,0.000540
50%,4.219090e+06,518.000000,0.001931
75%,4.329083e+06,1852.000000,0.007576
max,4.440001e+06,23674.000000,1.000000


In [72]:
val_predicitons_MMR.to_csv('../../Data/val_predicitons_MMR_tem2vec_bilstmv1.csv', index=False)

# Mean reciprocal rank = 0.018487